In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
from pathlib import Path
import os.path
import cv2
from PIL import Image 
from PIL import ImageEnhance
from skimage.io import imread
import matplotlib.pyplot as plt

import random, pathlib, warnings, itertools, math
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# parameter
dim = (100, 100)
channel = (3, )
input_shape = dim + channel

#ukuran batch 
batch_size = 32


In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1. / 255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

In [ ]:
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Vegetable/train/',
                                                    target_size=dim,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

val_generator = val_datagen.flow_from_directory('/content/drive/MyDrive/Vegetable/validation/',
                                                target_size=dim,
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                shuffle=True)

test_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/Vegetable/test/',
                                                  target_size=dim,
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

num_class = test_generator.num_classes
labels = train_generator.class_indices.keys()

Found 7500 images belonging to 15 classes.
Found 1500 images belonging to 15 classes.
Found 1500 images belonging to 15 classes.


In [ ]:
print(labels)

dict_keys(['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato'])


In [ ]:
def tf_data_generator(generator, input_shape):
    num_class = generator.num_classes
    tf_generator = tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([None
                        , input_shape[0]
                        , input_shape[1]
                        , input_shape[2]]
                       ,[None, num_class])
    )
    return tf_generator

In [ ]:
train_data = tf_data_generator(train_generator, input_shape)
test_data = tf_data_generator(test_generator, input_shape)
val_data = tf_data_generator(val_generator, input_shape)

In [ ]:
!pip install -U --pre efficientnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.3 MB/s 


In [ ]:
from efficientnet.tfkeras import EfficientNetB1

In [ ]:
from tensorflow.keras import layers,Sequential
from tensorflow.keras.models import Model

In [ ]:
def feature_extractorfeature_extractor(inputs):
    feature_extractor = EfficientNetB1(
                                      input_shape=input_shape,
                                      include_top=False,
                                      weights='noisy-student',
                                      classes=num_class,
                                  )(inputs)
    
    return feature_extractor

def classifier(inputs):
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(1024, activation="relu")(x)
    x = layers.Dropout(0.4) (x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dense(num_class, activation="softmax", name="classification")(x)
    
    return x

def final_model(inputs):
    efficientNet_feature_extractor = feature_extractorfeature_extractor(inputs)
    classification_output = classifier(efficientNet_feature_extractor)
    
    return classification_output

def define_compile_model():
    inputs = layers.Input(shape=input_shape)
    classification_output = final_model(inputs) 
    model = Model(inputs=inputs, outputs = classification_output)
    model.compile(optimizer='adam', 
                loss='categorical_crossentropy',
                metrics = ['accuracy'])
  
    return model

In [ ]:
model = define_compile_model()
clear_output()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 efficientnet-b1 (Functional  (None, 4, 4, 1280)       6575232   
 )                                                               
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0     

In [ ]:

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x=train_data,
        steps_per_epoch=len(train_generator),
        epochs=5,
        validation_data=val_data,
        validation_steps=len(val_generator), 
        shuffle=True,
        verbose = 1)

Epoch 1/5
235/235 [==============================] - 2119s 9s/step - loss: 0.7722 - accuracy: 0.7711 - val_loss: 0.5430 - val_accuracy: 0.8727
Epoch 2/5
235/235 [==============================] - 66s 281ms/step - loss: 0.2589 - accuracy: 0.9348 - val_loss: 0.1411 - val_accuracy: 0.9593
Epoch 3/5
235/235 [==============================] - 66s 282ms/step - loss: 0.2403 - accuracy: 0.9431 - val_loss: 0.1360 - val_accuracy: 0.9733
Epoch 4/5
235/235 [==============================] - 66s 282ms/step - loss: 0.1802 - accuracy: 0.9592 - val_loss: 0.1966 - val_accuracy: 0.9647
Epoch 5/5
235/235 [==============================] - 66s 280ms/step - loss: 0.1373 - accuracy: 0.9676 - val_loss: 0.1322 - val_accuracy: 0.9693


In [ ]:
model.save('model_13.h5')

In [ ]:
history_df = pd.DataFrame(r.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
history_df.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
loss, acc = model.evaluate(train_data, steps=len(train_generator), verbose=0)
print('Accuracy on training data: {:.4f} \nLoss on training data: {:.4f}'.format(acc,loss),'\n')
 
loss, acc = model.evaluate(test_data, steps=len(test_generator), verbose=0)
print('Accuracy on test data: {:.4f} \nLoss on test data: {:.4f}'.format(acc,loss),'\n')    

Accuracy on training data: 0.9671 
Loss on training data: 0.1388 

Accuracy on test data: 0.9647 
Loss on test data: 0.1563 



#Membuat prediksi gambar sayuran

In [ ]:
import requests
from io import BytesIO

from PIL import Image
import numpy as np

In [ ]:
# Parameters
input_size = (100,100)
channel = (3,)
input_shape = input_size + channel

#labelnya
labels = ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']

In [ ]:
def preprocess(img,input_size):
    nimg = img.convert('RGB').resize(input_size, resample= 0)
    img_arr = (np.array(nimg))/255
    return img_arr

def reshape(imgs_arr):
    return np.stack(imgs_arr, axis=0)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
MODEL_PATH = '/content/model_13.h5'
model = load_model(MODEL_PATH,compile=False)


In [ ]:
# read image
im = Image.open('/content/testsayuran2.jpg')#harusnya cucumber
X = preprocess(im,input_size)
X = reshape([X])
y = model.predict(X)

print( labels[np.argmax(y)], np.max(y) )

Cucumber 0.9614993


In [ ]:
# read image
im = Image.open('/content/testsayuran.jpg')#harusnya carrot
X = preprocess(im,input_size)
X = reshape([X])
y = model.predict(X)

print( labels[np.argmax(y)], np.max(y) )

Carrot 0.99999535


In [ ]:
# read image
im = Image.open('/content/testsayuran3.jpg')#harusnya tomato
X = preprocess(im,input_size)
X = reshape([X])
y = model.predict(X)

print( labels[np.argmax(y)], np.max(y) )

Capsicum 0.72162426


In [ ]:
# read image
im = Image.open('/content/testsayuran4.jpg')#harusnya capscium
X = preprocess(im,input_size)
X = reshape([X])
y = model.predict(X)

print( labels[np.argmax(y)], np.max(y) )

Capsicum 0.99998534


In [ ]:
y

array([[6.1786518e-09, 2.2857448e-10, 4.7307519e-10, 1.3648678e-06,
        9.0875345e-12, 1.9153655e-08, 9.9998534e-01, 1.4689846e-08,
        6.2856427e-09, 1.2002562e-09, 2.4512889e-07, 1.2223125e-09,
        1.6055598e-07, 2.6039626e-10, 1.2783145e-05]], dtype=float32)